<a href="https://colab.research.google.com/github/bsyh/foursquare/blob/main/embedding_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 团队版本
https://github.com/bsyh/foursquare


access to google drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
root_path = 'drive/MyDrive/Foursquare/'

Mounted at /content/drive


create path

In [ ]:
import subprocess, os, sys, ipykernel

def wget(url, outputdir):
  res = subprocess.run(['wget', url, '-P', f'{outputdir}'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(res)

def createPath(filepath):
  os.makedirs(filepath, exist_ok=True)

data_path = f'{root_path}/data'
createPath(data_path)

out_path = f'{root_path}/out'
createPath(out_path)


download dataset

In [ ]:


#@markdown If you don't want the dataset be downloaded, uncheck the box:
download_dataset = False #@param{type:"boolean"}

if download_dataset:
  !pip install gdown
  !cd f'{data_path}'
  !gdown --folder https://drive.google.com/drive/folders/1-8C3oY9dLErVL3zCteWayKg9x_ALjGlT

importimport packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train = pd.read_csv(f'{data_path}/train.csv')
test = pd.read_csv(f'{data_path}/test.csv')
pd.options.display.max_rows,pd.options.display.max_columns = 1000,14 # display all rows annd columns
train.head()

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,NaN,Bars,P_677e840bb6fc7e
1,E_000002eae2a589,Carioca Manero,-22.907225,-43.178244,NaN,NaN,NaN,NaN,BR,NaN,NaN,Brazilian Restaurants,P_d82910d8382a83
2,E_000007f24ebc95,ร้านตัดผมการาเกด,13.780813,100.484900,NaN,NaN,NaN,NaN,TH,NaN,NaN,Salons / Barbershops,P_b1066599e78477
3,E_000008a8ba4f48,Turkcell,37.844510,27.844202,Adnan Menderes Bulvarı,NaN,NaN,NaN,TR,NaN,NaN,Mobile Phone Shops,P_b2ed86905a4cd3
4,E_00001d92066153,Restaurante Casa Cofiño,43.338196,-4.326821,NaN,Caviedes,Cantabria,NaN,ES,NaN,NaN,Spanish Restaurants,P_809a884d4407fb


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138812 entries, 0 to 1138811
Data columns (total 13 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   id                 1138812 non-null  object 
 1   name               1138811 non-null  object 
 2   latitude           1138812 non-null  float64
 3   longitude          1138812 non-null  float64
 4   address            742191 non-null   object 
 5   city               839623 non-null   object 
 6   state              718226 non-null   object 
 7   zip                543386 non-null   object 
 8   country            1138801 non-null  object 
 9   url                267724 non-null   object 
 10  phone              342855 non-null   object 
 11  categories         1040505 non-null  object 
 12  point_of_interest  1138812 non-null  object 
dtypes: float64(2), object(11)
memory usage: 112.9+ MB


D:

In [ ]:
train_tiny_1 = pd.read_csv("drive/MyDrive/Foursquare/train_tiny_1.csv")
# test = pd.read_csv(f'{data_path}/test.csv')

In [ ]:
!pip install -U sentence-transformers
import sentence_transformers
from sentence_transformers import SentenceTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.0 MB/s 
     |████████████████████████████████| 4.4 MB 15.0 MB/s 
     |████████████████████████████████| 1.2 MB 49.3 MB/s 
     |████████████████████████████████| 101 kB 14.0 MB/s 
     |████████████████████████████████| 596 kB 64.0 MB/s 
     |████████████████████████████████| 6.6 MB 32.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=63440f82cc5ae1eb480a49475868225377868319b0de7f986d621eae0e545ccb
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
train_tiny_1 = train_tiny_1.fillna("")
text = train_tiny_1["name"]+ '[sep]' + train_tiny_1['address'] + '[sep]' + train_tiny_1['city'] + '[sep]' + train_tiny_1['state'] + '[sep]' + train_tiny_1['country'] + '[sep]' + train_tiny_1['url'] + '[sep]' + train_tiny_1['categories']
text

0      Café Stad Oudenaarde[sep]Abdijstraat[sep]Neder...
1      Carioca Manero[sep][sep][sep][sep]BR[sep][sep]...
2      ร้านตัดผมการาเกด[sep][sep][sep][sep]TH[sep][se...
3      Turkcell[sep]Adnan Menderes Bulvarı[sep][sep][...
4      Restaurante Casa Cofiño[sep][sep]Caviedes[sep]...
                             ...                        
217    Fróðskaparsetur Føroya[sep]Nóatún[sep]Torshavn...
218    Monte Roraima[sep][sep]Uiramutã[sep]Roraima[se...
219    La Estrella de Panamá[sep]Vía Frangipani[sep]C...
220    Juba Grand Hotel[sep]Jubatown[sep]Juba[sep]Cen...
221    Infantes[sep][sep][sep][sep]VU[sep][sep]Indian...
Length: 222, dtype: object

In [ ]:
tmp = text.tolist()
model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')
embeddings = model.encode(tmp)
for sentence, embedding in zip(tmp, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

流式输出内容被截断，只能显示最后 5000 行内容。
  7.12045953e-02 -1.23569094e-01  1.24613732e-01 -1.22444600e-01
 -3.71176787e-02  9.20786783e-02  2.33387500e-02  1.65047362e-01
  1.76392481e-01  4.16992873e-01 -4.34166454e-02  1.71420261e-01
  1.31529234e-02  2.97779329e-02  2.32752964e-01  3.61506015e-01
  7.55415633e-02 -1.25049308e-01 -2.66162366e-01  4.11763936e-02
 -2.40571983e-03  1.15447521e-01  1.58989459e-01 -4.99878898e-02
 -1.90402120e-02  1.92167997e-01  1.05177522e-01  4.67441045e-02
  1.71418503e-01  2.89397746e-01 -4.56742533e-02  2.90079117e-01
 -2.77632717e-02 -4.97802161e-02  1.82784900e-01 -1.21966571e-01
 -8.92129317e-02  1.95392538e-02  1.65498015e-02 -1.70809224e-01
  8.11125189e-02 -1.41296536e-01  1.59902312e-02 -2.84730103e-02
  7.90098667e-01 -1.94265485e-01  6.63017407e-02  1.26775622e-01
  4.79766846e-01 -1.49978255e-03  6.46033734e-02 -1.60553575e-01
 -2.17796892e-01 -6.60983920e-02  1.17947608e-01  2.72665232e-01
  7.24182799e-02  1.46675080e-01  1.68568522e-01 -6.17003143e-0

# Data preprocessing & Feature transformation:

1. location (latitude, longtitude): finding the nearest several points accroding to distance.
2. group of words (name, address, categories): lowercasing, ...


In [ ]:
# 1. location
# calculating nearest points accroding to distance
neighbor = NearestNeighbors(n_neighbors=2, metric="haversine", n_jobs=-1) # haversine: 2 arcsin(sqrt(sin^2(0.5*dx) + cos(x1)cos(x2)sin^2(0.5*dy)))
position = train.iloc[:,2:4]
neighbor.fit(position)
distance = neighbor.kneighbors(position)[0]
cloest_pos = neighbor.kneighbors(position)[1]

This notebook shows how to solve the problem as a multi-class classification by finding candidate points based on geographic location.
Similarity as a string, such as edit distance and LCS (Longest Common Subsequence), was used for the features of the candidate points.

Inference is made on test data only, but the code for training is left commented out.

In addition, making the matches bidirectional as a post-processing step improved the score by about 1%.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
import gc
import random
from glob import glob
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import warnings
import seaborn as sns
import pickle
import json
import re
import time
import sys
from requests import get
import multiprocessing
import joblib

class CFG:
    seed = 46
    target = "point_of_interest"
    n_neighbors = 10
    n_splits = 3

    expID = ""
    if "google.colab" in sys.modules:
        expID = get("http://172.28.0.2:9000/api/sessions").json()[0]["name"].split(".")[0]

random.seed(CFG.seed)
os.environ["PYTHONHASHSEED"] = str(CFG.seed)
np.random.seed(CFG.seed)

plt.rcParams["font.size"] = 13
warnings.filterwarnings('ignore')

# %cd /content/drive/MyDrive/kaggle/foursquare-location-matching/{CFG.expID}


In [ ]:

train = pd.read_csv("drive/MyDrive/Foursquare/train.csv")
test = pd.read_csv("drive/MyDrive/Foursquare/test.csv")
test[CFG.target] = "TEST"

train.head(1)

FileNotFoundError: ignored

# New Section

# New Section

Devide Train Data into about 600K×2

In [ ]:
kf = GroupKFold(n_splits=2)
for i, (trn_idx, val_idx) in enumerate(kf.split(train, train[CFG.target], train[CFG.target])):
    train.loc[val_idx, "set"] = i
train["set"].value_counts()

Search Candidates

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

def add_neighbor_features(df):
    dfs = []
    columns = ['id', 'name', 'address', 'city', 'state',
           'zip', 'country', 'url', 'phone', 'categories']
    for c in columns:
        if c != "id":
            df[c] = df[c].astype(str).str.lower()

    for country, country_df in tqdm(df.groupby("country")):
        country_df = country_df.reset_index(drop=True)
        
        knn = KNeighborsRegressor(n_neighbors=min(len(country_df), CFG.n_neighbors), 
                                  metric='haversine', n_jobs=-1)
        knn.fit(country_df[['latitude','longitude']], country_df.index)
        dists, nears = knn.kneighbors(country_df[['latitude','longitude']], return_distance=True)

        targets = country_df[CFG.target].values
        for i in range(min(len(country_df), CFG.n_neighbors)):
            country_df[f"d_near_{i}"] = dists[:, i]
            country_df[f"near_target_{i}"] = targets[nears[:, i]]
            for c in columns:
                country_df[f"near_{c}_{i}"] = country_df[c].values[nears[:, i]]

        for i in range(min(len(country_df), CFG.n_neighbors), CFG.n_neighbors):
            country_df[f"d_near_{i}"] = np.nan
            country_df[f"near_target_{i}"] = np.nan
            for c in columns:
                country_df[f"near_{c}_{i}"] = np.nan

        dfs.append(country_df)
    df = pd.concat(dfs).reset_index(drop=True)
    return df

train = pd.concat([
    add_neighbor_features(train[train["set"]==0]), 
    add_neighbor_features(train[train["set"]==1]), 
])
test = add_neighbor_features(test)

train.head(1)

Create Target

In [ ]:
for i in range(CFG.n_neighbors):
    train.loc[train[CFG.target]==train[f"near_target_{i}"], "target"] = i
train.head()

Check Maximum Score

In [ ]:
# https://www.kaggle.com/code/columbia2131/foursquare-iou-metrics
def get_id2poi(input_df: pd.DataFrame) -> dict:
    return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
    return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

def get_score(input_df: pd.DataFrame):
    scores = []
    for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
        targets = poi2ids[id2poi[id_str]]
        preds = set(matches.split())
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()

id2poi = get_id2poi(train)
poi2ids = get_poi2ids(train)

In [ ]:
scores = []

train["matches"] = ""
for i in tqdm(range(CFG.n_neighbors)):
    idx = train[CFG.target]==train[f"near_target_{i}"]
    train.loc[idx, "matches"] += " " + train.loc[idx, f"near_id_{i}"]
    scores.append(get_score(train))
train["mathces"] = None

In [ ]:
plt.subplots(figsize=(8, 3), facecolor="white")
plt.plot(range(CFG.n_neighbors), scores, marker="o")
plt.grid()
plt.xlabel("# of candidates")
plt.ylabel("Maximum Score")
plt.ylim([0.6, 1.0])
plt.show()

In [ ]:
del train
gc.collect()

# Catboost

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from catboost import CatBoostClassifier
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    return summary

In [ ]:
!ls

drive  sample_data
